<a href="https://colab.research.google.com/github/RickBarretto/llm-playground/blob/main/notebooks/tucano-2b4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Author: [TucanoBR at Huggingface](https://huggingface.co/TucanoBR)
- Source: [Chat UI for Tucano](https://colab.research.google.com/drive/1fEW10CXksMfMv1veLr22OESwDs6e-W1b)


In [ ]:
!pip install -q gradio

import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

model_id="TucanoBR/Tucano-2b4-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="sdpa" if torch.cuda.is_available() else "eager"
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [2]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def predict(message, history):
    stop = StopOnTokens()
    message = "<instruction>" + message + "</instruction>"
    model_inputs = tokenizer([message], return_tensors="pt").to(device)
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        repetition_penalty=1.2,
        max_new_tokens=1024,
        do_sample=True,
        top_p=1.,
        top_k=50,
        temperature=0.1,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '</s>' in partial_message:
            break
        yield partial_message

gr.ChatInterface(predict,
                 title="Tucano 🦜",
                 description="Faça uma pergunta para o Tucano",
                 examples=['Qual a capital do Rio Grande do Sul?', 'Invente uma história sobre um garoto chamado Pelé.']
                 ).launch()